In [48]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/feature_store/gapic-feature-store.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/official/feature_store/gapic-feature-store.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This Colab introduces Vertex AI Feature Store, a managed cloud service for machine learning engineers and data scientists to store, serve, manage and share machine learning features at a large scale.

This Colab assumes that you understand basic Google Cloud concepts such as [Project](https://cloud.google.com/storage/docs/projects), [Storage](https://cloud.google.com/storage) and [Vertex AI](https://cloud.google.com/vertex-ai/docs). Some machine learning knowledge is also helpful but not required.

### Dataset

This Colab uses a movie recommendation dataset as an example throughout all the sessions. The task is to train a model to predict if a user is going to watch a movie and serve this model online. 

### Objective

In this notebook, you will learn how to:

    * How to import your features into Vertex AI Feature Store.
    * How to serve online prediction requests using the imported features.
    * How to access imported features in offline jobs, such as training jobs.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Cloud Bigtable

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

**If you are using Colab or Google Cloud Notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

## Before you begin

### Select a GPU runtime

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select "Runtime --> Change runtime type > GPU"**

### Install additional packages

For this Colab, you need the Vertex SDK for Python.

In [1]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

In [2]:
! pip install {USER_FLAG} --upgrade git+https://github.com/googleapis/python-aiplatform.git@main


  Cloning https://github.com/googleapis/python-aiplatform.git (to revision main) to /tmp/pip-req-build-pdtciku4
  Running command git clone -q https://github.com/googleapis/python-aiplatform.git /tmp/pip-req-build-pdtciku4


### Restart the kernel

After you install the SDK, you need to restart the notebook kernel so it can find the packages. You can restart kernel from *Kernel -> Restart Kernel*, or running the following:

In [3]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [1]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  


Otherwise, set your project ID here.

In [2]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "intro-to-gcp-331822"  # @param {type:"string"}
print("Project ID: ", PROJECT_ID)

Project ID:  intro-to-gcp-331822


### Authenticate your Google Cloud account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [3]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# If on Google Cloud Notebooks, then don't execute this code
if not IS_GOOGLE_CLOUD_NOTEBOOK:
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

## Prepare for output

### Step 1. Create dataset for output

You need a BigQuery dataset to host the output data in `us-central1`. Input the name of the dataset you want to created and specify the name of the table you want to store the output later. These will be used later in the notebook.

**Make sure that the table name does NOT already exist**.


In [4]:
from datetime import datetime
from google.cloud import bigquery

In [5]:
# Output dataset
DESTINATION_DATA_SET = "movie_predictions"  # @param {type:"string"}
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
DESTINATION_DATA_SET = "{prefix}_{timestamp}".format(
    prefix=DESTINATION_DATA_SET, timestamp=TIMESTAMP
)

# Output table. Make sure that the table does NOT already exist; the BatchReadFeatureValues API cannot overwrite an existing table
DESTINATION_TABLE_NAME = "training_data"  # @param {type:"string"}

DESTINATION_PATTERN = "bq://{project}.{dataset}.{table}"
DESTINATION_TABLE_URI = DESTINATION_PATTERN.format(
    project=PROJECT_ID, dataset=DESTINATION_DATA_SET, table=DESTINATION_TABLE_NAME
)

In [6]:
# Create dataset
REGION = "us-central1"  # @param {type:"string"}
client = bigquery.Client(project=PROJECT_ID)
dataset_id = "{}.{}".format(client.project, DESTINATION_DATA_SET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = client.create_dataset(dataset)
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset intro-to-gcp-331822.movie_predictions_20220211202636


### Import libraries and define constants

In [7]:
from google.cloud import aiplatform
from google.cloud.aiplatform import Featurestore, EntityType, Feature
LOCATION = 'us-central1'
FEATURESTORE_ID = "movie_prediction"
INPUT_CSV_FILE = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movie_prediction_sdk.csv"
ONLINE_STORE_FIXED_NODE_COUNT = 1
SYNC = True
FORCE = True

aiplatform.init(project=PROJECT_ID, location=LOCATION)

## Terminology and Concept

### Featurestore Data model

Vertex AI Feature Store organizes data with the following 3 important hierarchical concepts:
```
Featurestore -> EntityType -> Feature
```
* **Featurestore**: the place to store your features
* **EntityType**: under a Featurestore, an *EntityType* describes an object to be modeled, real one or virtual one.
* **Feature**: under an EntityType, a *feature* describes an attribute of the EntityType

In the movie prediction example, you will create a featurestore called *movie_prediction*. This store has 2 entity types: *Users* and *Movies*. The Users entity type has the age, gender, and liked_genres features. The Movies entity type has the genres and average rating features.


## Create Featurestore and Define Schemas

### Create Featurestore

The method to create a Featurestore returns a
[long-running operation](https://google.aip.dev/151) (LRO). An LRO starts an asynchronous job. LROs are returned for other API
methods too, such as updating or deleting a featurestore. Running the code cell will create a featurestore and print the process log.

In [8]:
# Feature Store imports & SDK create
from google.cloud.aiplatform import Featurestore, EntityType, Feature
FEATURESTORE_ID = "movie_prediction"
fs = Featurestore.create(
        project=PROJECT_ID,
        location=LOCATION,
        featurestore_id=FEATURESTORE_ID,
        online_store_fixed_node_count=ONLINE_STORE_FIXED_NODE_COUNT,
        sync=SYNC,
)

INFO:google.cloud.aiplatform.featurestore.featurestore:Creating Featurestore
INFO:google.cloud.aiplatform.featurestore.featurestore:Create Featurestore backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/operations/2038521564651061248
INFO:google.cloud.aiplatform.featurestore.featurestore:Featurestore created. Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction
INFO:google.cloud.aiplatform.featurestore.featurestore:To use this Featurestore in another session:
INFO:google.cloud.aiplatform.featurestore.featurestore:featurestore = aiplatform.Featurestore('projects/455082776678/locations/us-central1/featurestores/movie_prediction')


You can use the function call below to retrieve a Featurestore and check that it has been created.


In [9]:
fs = Featurestore(
        project=PROJECT_ID,
        location=LOCATION,
        featurestore_name=FEATURESTORE_ID,
)
print(fs.gca_resource)

name: "projects/455082776678/locations/us-central1/featurestores/movie_prediction"
create_time {
  seconds: 1644611218
  nanos: 82998000
}
update_time {
  seconds: 1644611218
  nanos: 131770000
}
etag: "AMEw9yN0DB5YJZ_Ryg0iPuJBVJFudKOn_L24feCwm8jjXMdWPHyDX6sPvmf4S2W-L5E="
online_serving_config {
  fixed_node_count: 1
}
state: STABLE



### Create Entity Type

Entity types can be created within the Featurestore class. Below we create the Users entity type and Movies entity type.

In [10]:
# Create users entity type
users_entity_type = fs.create_entity_type(
    entity_type_id="users",
    description="Users entity",
    sync=SYNC
)

INFO:google.cloud.aiplatform.featurestore.entity_type:Creating EntityType
INFO:google.cloud.aiplatform.featurestore.entity_type:Create EntityType backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/operations/6650207583078449152
INFO:google.cloud.aiplatform.featurestore.entity_type:EntityType created. Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users
INFO:google.cloud.aiplatform.featurestore.entity_type:To use this EntityType in another session:
INFO:google.cloud.aiplatform.featurestore.entity_type:entity_type = aiplatform.EntityType('projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users')


In [21]:
# Create movies entity type
movies_entity_type = fs.create_entity_type(
    entity_type_id="movies",
    description="Movies entity",
    sync=SYNC
)

INFO:google.cloud.aiplatform.featurestore.entity_type:Creating EntityType
INFO:google.cloud.aiplatform.featurestore.entity_type:Create EntityType backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/operations/8638546818562523136
INFO:google.cloud.aiplatform.featurestore.entity_type:EntityType created. Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies
INFO:google.cloud.aiplatform.featurestore.entity_type:To use this EntityType in another session:
INFO:google.cloud.aiplatform.featurestore.entity_type:entity_type = aiplatform.EntityType('projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies')


To retrieve an entity type or check that it has been created use the [get_entity_type](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L106) or [list_entity_types](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L278) methods on the Featurestore object.



In [12]:
users_entity_type = fs.get_entity_type(entity_type_id='users')
movies_entity_type = fs.get_entity_type(entity_type_id='movies')
print(users_entity_type)
print(movies_entity_type)

resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users
resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies


In [14]:
fs.list_entity_types()

 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users]

Feature [monitoring](https://cloud.google.com/vertex-ai/docs/featurestore/monitoring) is in preview, so you need to use v1beta1 Python. The easiest way to set this for now is using [console UI](https://console.cloud.google.com/vertex-ai/features). For completeness, below is example to do this using v1beta1 SDK


In [15]:
API_ENDPOINT = "us-central1-aiplatform.googleapis.com"  # @param {type:"string"}

In [16]:
from google.cloud.aiplatform_v1beta1 import \
    FeaturestoreServiceClient as v1beta1_FeaturestoreServiceClient
from google.cloud.aiplatform_v1beta1.types import \
    entity_type as v1beta1_entity_type_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_monitoring as v1beta1_featurestore_monitoring_pb2
from google.cloud.aiplatform_v1beta1.types import \
    featurestore_service as v1beta1_featurestore_service_pb2
from google.protobuf.duration_pb2 import Duration

v1beta1_admin_client = v1beta1_FeaturestoreServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

# Enable monitoring for users entity type.
# All Features belonging to this EntityType will by default inherit the monitoring config.
v1beta1_admin_client.update_entity_type(
    v1beta1_featurestore_service_pb2.UpdateEntityTypeRequest(
        entity_type=v1beta1_entity_type_pb2.EntityType(
            name=v1beta1_admin_client.entity_type_path(
                PROJECT_ID, REGION, FEATURESTORE_ID, "users"
            ),
            monitoring_config=v1beta1_featurestore_monitoring_pb2.FeaturestoreMonitoringConfig(
                snapshot_analysis=v1beta1_featurestore_monitoring_pb2.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                    monitoring_interval=Duration(seconds=86400),  # 1 day
                ),
            ),
        ),
    )
)

name: "projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users"
description: "Users entity"
create_time {
  seconds: 1644611232
  nanos: 831059000
}
update_time {
  seconds: 1644611264
  nanos: 977207000
}
etag: "AMEw9yOQSc1mz-WnACSrJgV64-L8feKW9Z8jONenGaNFEabC_jHa3dCcjQIM9lzlLbVm"
monitoring_config {
  snapshot_analysis {
    monitoring_interval {
      seconds: 86400
    }
    monitoring_interval_days: 1
  }
}


### Create Feature
Features can be created within each entity_type. Below we add defining features to the Users entity type and Movies entity type.

In [17]:
# to create features one at a time use
users_feature_age = users_entity_type.create_feature(
    feature_id="age",
    value_type="INT64",
    description="User age",
    labels={"status": "passed"}
)

users_feature_gender = users_entity_type.create_feature(
    feature_id="gender",
    value_type="STRING",
    description="User gender",
    labels={"status": "passed"}
)

users_feature_liked_genres = users_entity_type.create_feature(
    feature_id="liked_genres",
    value_type="STRING_ARRAY",
    description="An array of genres this user liked",
    labels={"status": "passed"}
)

INFO:google.cloud.aiplatform.featurestore.feature:Creating Feature
INFO:google.cloud.aiplatform.featurestore.feature:Create Feature backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/operations/5956653240463392768
INFO:google.cloud.aiplatform.featurestore.feature:Feature created. Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/age
INFO:google.cloud.aiplatform.featurestore.feature:To use this Feature in another session:
INFO:google.cloud.aiplatform.featurestore.feature:feature = aiplatform.Feature('projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/age')
INFO:google.cloud.aiplatform.featurestore.feature:Creating Feature
INFO:google.cloud.aiplatform.featurestore.feature:Create Feature backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/operations/2241183547882733568
INFO:google.cloud.aiplatform.fea

Use the [list_features](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/entity_type.py#L349) method to list all the features of a given entity type.

In [18]:
users_entity_type.list_features()

 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/age,
 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/gender,
 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/liked_genres]

In [19]:
movies_feature_configs = {
    "title": {
        "value_type": "STRING",
        "description" : "The title of the movie",
        "labels": {"status": "passed"}
        },
    "genres": {
        "value_type": "STRING",
        "description" : "The genre of the movie",
        "labels": {"status": "passed"}
        },
    "average_rating": {
        "value_type": "DOUBLE",
        "description" : "The average rating for the movie, range is [1.0-5.0]",
        "labels": {"status": "passed"}
        },
}

In [22]:
movie_features = movies_entity_type.batch_create_features(
        feature_configs = movies_feature_configs,
        sync=True)

INFO:google.cloud.aiplatform.featurestore.entity_type:Batch creating features EntityType entityType: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies
INFO:google.cloud.aiplatform.featurestore.entity_type:Batch create Features EntityType entityType backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/operations/7505891512278843392
INFO:google.cloud.aiplatform.featurestore.entity_type:EntityType entityType Batch created features. Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies


In [23]:
# listing all the features in the featurestore
for entity_type in fs.list_entity_types():
  for feature in entity_type.list_features():
    print(feature)

resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/title
resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/average_rating
resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/genres
resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/age
resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/gender
resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/liked_genres


## Search created features

While the `list_features` method allows you to easily view all features of a single
entity type, the [search](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/feature.py#L352) method in the Feature class searches across all featurestores and entity types in a given location (such as `us-central1`) and returns a list of features. This can help you discover features that were created by someone else.

You can query based on feature properties including feature ID, entity type ID,
and feature description. You can also limit results by filtering on a specific
featurestore, feature value type, and/or labels.

In [24]:
my_features = Feature.search(query="featurestore_id={}".format(FEATURESTORE_ID))
my_features

 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/average_rating,
 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/genres,
 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/title,
 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/age,
 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/gender,
 resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/features/liked_genres]

Now, narrow down the search to features that are of type `DOUBLE`

In [25]:
# Search for all features with value type `DOUBLE`
double_features = Feature.search(query="value_type=DOUBLE AND featurestore_id={}".format(FEATURESTORE_ID))
double_features[0].gca_resource

name: "projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/average_rating"
description: "The average rating for the movie, range is [1.0-5.0]"
value_type: DOUBLE
create_time {
  seconds: 1644611388
  nanos: 881632000
}
update_time {
  seconds: 1644611388
  nanos: 881632000
}
labels {
  key: "status"
  value: "passed"
}

Or, limit the search results to features with specific keywords in their ID and type.

In [26]:
# Filter on feature value type and keywords.
title_features = Feature.search(query="feature_id:title AND value_type=STRING AND featurestore_id={}".format(FEATURESTORE_ID))
title_features[0].gca_resource

name: "projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/features/title"
description: "The title of the movie"
value_type: STRING
create_time {
  seconds: 1644611388
  nanos: 877492000
}
update_time {
  seconds: 1644611388
  nanos: 877492000
}
labels {
  key: "status"
  value: "passed"
}

## Import Feature Values

You need to import feature values before you can use them for online/offline serving. In this step, you will learn how to import feature values by ingesting the values from GCS (Google Cloud Storage). We can also import feature values from BigQuery or a Pandas dataframe.


### Source Data Format and Layout

As mentioned above, BigQuery table/Avro/CSV are supported. No matter what format you are using, each imported entity *must* have an ID; also, each entity can *optionally* have a timestamp, specifying when the feature values are generated. This Colab uses Avro as an input, located at this public [bucket](https://pantheon.corp.google.com/storage/browser/cloud-samples-data-us-central1/vertex-ai/feature-store/datasets). The Avro schemas are as follows:

**For the Users entity**:
```
schema = {
  "type": "record",
  "name": "User",
  "fields": [
      {
       "name":"user_id",
       "type":["null","string"]
      },
      {
       "name":"age",
       "type":["null","long"]
      },
      {
       "name":"gender",
       "type":["null","string"]
      },
      {
       "name":"liked_genres",
       "type":{"type":"array","items":"string"}
      },
      {
       "name":"update_time",
       "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
      },
  ]
 }
```

**For the Movies entity**
```
schema = {
 "type": "record",
 "name": "Movie",
 "fields": [
     {
      "name":"movie_id",
      "type":["null","string"]
     },
     {
      "name":"average_rating",
      "type":["null","double"]
     },
     {
      "name":"title",
      "type":["null","string"]
     },
     {
      "name":"genres",
      "type":["null","string"]
     },
     {
      "name":"update_time",
      "type":["null",{"type":"long","logicalType":"timestamp-micros"}]
     },
 ]
}
```

### Import feature values for Users

When importing, specify the following in your request:

*   IDs of the features to import 
*   Data source URL
*   Data source format: BigQuery Table/Avro/CSV


In [27]:
USERS_FEATURES_IDS = [feature.name for feature in users_entity_type.list_features()]
USERS_FEATURE_TIME = "update_time"
USERS_ENTITY_ID_FIELD = "user_id"
USERS_GCS_SOURCE_URI = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/users.avro"
GCS_SOURCE_TYPE = "avro"
WORKER_COUNT = 1
print(USERS_FEATURES_IDS)

['age', 'gender', 'liked_genres']


In [28]:
users_entity_type.ingest_from_gcs(
    feature_ids=USERS_FEATURES_IDS,
    feature_time=USERS_FEATURE_TIME,
    entity_id_field=USERS_ENTITY_ID_FIELD,
    gcs_source_uris=USERS_GCS_SOURCE_URI,
    gcs_source_type=GCS_SOURCE_TYPE,
    worker_count=WORKER_COUNT,
    )

INFO:google.cloud.aiplatform.featurestore.entity_type:Importing EntityType feature values: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users
INFO:google.cloud.aiplatform.featurestore.entity_type:Import EntityType feature values backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users/operations/2101571959434248192
INFO:google.cloud.aiplatform.featurestore.entity_type:EntityType feature values imported. Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users


resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/users

### Import feature values for Movies

Similarly, import feature values for 'movies' into the featurestore.


In [29]:
MOVIES_FEATURES_IDS = [feature.name for feature in movies_entity_type.list_features()]
MOVIES_FEATURE_TIME = "update_time"
MOVIES_ENTITY_ID_FIELD = "movie_id"
MOVIES_GCS_SOURCE_URI = "gs://cloud-samples-data-us-central1/vertex-ai/feature-store/datasets/movies.avro"
GCS_SOURCE_TYPE = "avro"
WORKER_COUNT = 1
print(MOVIES_FEATURES_IDS)

['title', 'average_rating', 'genres']


In [30]:
movies_entity_type.ingest_from_gcs(
    feature_ids=MOVIES_FEATURES_IDS,
    feature_time=MOVIES_FEATURE_TIME,
    entity_id_field=MOVIES_ENTITY_ID_FIELD,
    gcs_source_uris=MOVIES_GCS_SOURCE_URI,
    gcs_source_type=GCS_SOURCE_TYPE,
    worker_count=WORKER_COUNT,
    )

INFO:google.cloud.aiplatform.featurestore.entity_type:Importing EntityType feature values: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies
INFO:google.cloud.aiplatform.featurestore.entity_type:Import EntityType feature values backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies/operations/7397805121221951488
INFO:google.cloud.aiplatform.featurestore.entity_type:EntityType feature values imported. Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies


resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction/entityTypes/movies

## Online serving

[Online serving](https://cloud.google.com/vertex-ai/docs/featurestore/serving-online)
lets you serve feature values for small batches of entities. It's designed for latency-sensitive service, such as online model prediction. For example, for a movie service, you might want to quickly shows movies that the current user would most likely watch by using online predictions.

### Read one entity per request

With the Python SDK, it is easy to read feature values of one entity. By default, the SDK will return the  latest value of each feature, meaning the feature values with the most recent  timestamp.

To read feature values, specify the entity type ID and features to read. By default all the features of an entity type will be selected. The response will contain the title of each selected feature and display the feature values.

In [31]:
users_entity_type.read(
    entity_ids="bob"
)

,entity_id,gender,age,liked_genres
0,bob,Male,35,"[Action, Adventure]"


In [32]:
movies_entity_type.read(entity_ids='movie_01')

,entity_id,average_rating,genres,title
0,movie_01,4.9,Drama,The Shawshank Redemption


### Read multiple entities per request

To read feature values from multiple entities, specify the different entities. By default all the features of an entity type will be selected. Note that fetching only a small number of entities is recommended when using this SDK due to its latency-sensitive nature.

In [33]:
users_entity_type.read(
    entity_ids=["bob", "alice"]
)

,entity_id,age,gender,liked_genres
0,alice,55,Female,[Drama]
1,bob,35,Male,"[Action, Adventure]"


In [39]:
movies_entity_type.read(
    entity_ids=["movie_02", "movie_03", "movie_04"]
)

,entity_id,genres,title,average_rating
0,movie_02,Horror,The Shining,4.2
1,movie_03,Romance,Cinema Paradiso,4.5
2,movie_04,Action,The Dark Knight,4.6


Now that you have learned how to fetch imported feature values for online serving, the next step is learning how to use imported feature values for offline use cases.

## Batch Serving

Batch Serving is used to fetch a large batch of feature values for high-throughput, typically for training a model or batch prediction. In this section, you will learn how to prepare for training examples by using the Featurestore's batch serve function.

### Use case

**The task** is to prepare a training dataset to train a model, which predicts if a given user will watch a given movie. To achieve this, you need 2 sets of input:

*   Features: you already imported into the featurestore.
*   Labels: the groud-truth data recorded that user X has watched movie Y.


To be more specific, the ground-truth observation is described in Table 1 and the desired training dataset is described in Table 2. Each row in Table 2 is a result of joining the imported feature values from Vertex AI Feature Store according to the entity IDs and timestamps in Table 1. In this example,  the `age`, `gender` and `liked_genres` features from `users` and
the `titles`, `genres` and `average_rating` features from `movies` are chosen to train the model. Note that only positive examples are shown in these 2 tables, i.e., you can imagine there is a label column whose values are all `True`.

[batch_serve_to_bq](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/featurestore/featurestore.py#L770) takes Table 1 as
input, joins all required feature values from the featurestore, and returns Table 2 for training.

<h4 align="center">Table 1. Ground-truth Data</h4>

users | movies | timestamp            
----- | -------- | -------------------- 
alice  | Cinema Paradiso     | 2019-11-01T00:00:00Z 
bob  | The Shining     | 2019-11-15T18:09:43Z 
...   | ...      | ...     


<h4 align="center">Table 2. Expected Training Data Generated by Batch Read API (Positive Samples)</h4>

timestamp            | entity_type_users | age | gender | liked_genres | entity_type_movies | title | genre | average_rating  
-------------------- | ----------------- | --------------- | ---------------- | -------------------- | - | -------- | --------- | ----- 
2019-11-01T00:00:00Z | bob              | 35        | M                | [Action, Crime]      | movie_02            | The Shining | Horror | 4.8 
2019-11-01T00:00:00Z | alice             | 55        | F                | [Drama, Comedy]      | movie_03          | Cinema Paradiso | Romance | 4.5 | 
... | ... | ... | ... | ... | ... | ... | ... | ...
             

#### Why timestamp?

Note that there is a `timestamp` column in Table 2. This indicates the time when the ground-truth was observed. This is to avoid data inconsistency.

For example, the 1st row of Table 2 indicates that user `alice` watched movie `Cinema Paradiso` on `2019-11-01T00:00:00Z`. The featurestore keeps feature values for all timestamps but fetches feature values *only* at the given timestamp during batch serving. On 2019-11-01 alice might be 54 years old, but now alice might be 56; featurestore returns `age=54` as alice's age, instead of `age=56`, because that is the value of the feature at the observation time. Similarly, other features might be time-variant as well, such as liked_genres.

### Batch Read Feature Values

Assemble the request which specify the following info:

*   Where is the label data, i.e., Table 1.
*   Which features are read, i.e., the column names in Table 2.

The output is stored in a BigQuery table.


In [35]:
SERVING_FEATURE_IDS = {
    # to choose all the features use 'entity_type_id: ['*']'
    'users': ['age', 'gender', 'liked_genres'],
    'movies': ['title','average_rating', 'genres']
}

In [38]:
fs.batch_serve_to_bq(
    bq_destination_output_uri=DESTINATION_TABLE_URI,
    serving_feature_ids=SERVING_FEATURE_IDS,
    read_instances_uri=INPUT_CSV_FILE
)

INFO:google.cloud.aiplatform.featurestore.featurestore:Serving Featurestore feature values: projects/455082776678/locations/us-central1/featurestores/movie_prediction
INFO:google.cloud.aiplatform.featurestore.featurestore:Serve Featurestore feature values backing LRO: projects/455082776678/locations/us-central1/featurestores/movie_prediction/operations/5281113296357818368
INFO:google.cloud.aiplatform.featurestore.featurestore:Featurestore feature values served. Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction


resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction

After the LRO finishes, you should be able to see the result in the [BigQuery console](https://console.cloud.google.com/bigquery), as a new table under the BigQuery dataset created in the beginning.

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

You can also keep the project but delete the featurestore and the BigQuery dataset by running the code below:

In [45]:
# Featurestore delete
fs = Featurestore(
        featurestore_name=FEATURESTORE_ID,
        project=PROJECT_ID,
        location=LOCATION,

)
fs.delete(force=True)

INFO:google.cloud.aiplatform.featurestore.featurestore:Deleting Featurestore : projects/455082776678/locations/us-central1/featurestores/movie_prediction
INFO:google.cloud.aiplatform.featurestore.featurestore:Delete Featurestore  backing LRO: projects/455082776678/locations/us-central1/operations/6425027601709924352
INFO:google.cloud.aiplatform.featurestore.featurestore:Featurestore deleted. . Resource name: projects/455082776678/locations/us-central1/featurestores/movie_prediction


In [47]:
# Big Query delete
client = bigquery.Client(project=PROJECT_ID)
client.delete_dataset(
    DESTINATION_DATA_SET, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(DESTINATION_DATA_SET))

Deleted dataset 'movie_predictions_20220211194459'.
